In [ ]:
!pip install transformers datasets trl


In [ ]:
import pandas as pd

df=pd.read_csv("math_riddles.csv")
df.head()



,Riddle,Solution
0,What number becomes zero when you subtract 15 ...,30
1,I am a two-digit number. My tens digit is twic...,24
2,I am a three-digit number. My tens digit is 5 ...,142
3,A father is 30 years older than his son. In 10...,10
4,"If you multiply me by 4 and subtract 6, you ge...",6


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
tokenizer.pad_token=tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [ ]:
!pip install wandb

In [ ]:
from datasets import Dataset

dataset=Dataset.from_pandas(df)
dataset={"train":dataset}
dataset

{'train': Dataset({
     features: ['Riddle', 'Solution'],
     num_rows: 30
 })}

In [ ]:
def preprocess_data(examples):
    texts=[
        f"Riddle: {riddle} Solution: {solution}"
        for riddle,solution in zip(examples["Riddle"],examples["Solution"])
    ]
    return tokenizer(texts,padding="max_length",truncation=True,return_tensors="pt")

In [ ]:
tokenised_data=dataset["train"].map(preprocess_data,batched=True,remove_columns=["Riddle", "Solution"])
tokenised_data

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 30
})

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_arguments=TrainingArguments(
 output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=7,
    per_device_train_batch_size=2,
    evaluation_strategy="no",
    eval_steps=20,
    logging_steps=10,
    save_steps=20,
    learning_rate=1e-5,
    weight_decay=0.01,
    report_to=["wandb"],          # Enables wandb logging
    run_name="math-riddles-gpt2",  # Sets a name for the wandb run
    logging_dir="./logs"
)


trainer=SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenised_data,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-124-4b85877129e3>:21: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer=SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/30 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
10,4.079100
20,3.317300
30,2.807600
40,2.548700
50,2.328700
60,2.368200
70,2.256800
80,2.001300
90,2.171200
100,2.064900


TrainOutput(global_step=105, training_loss=2.568669627961658, metrics={'train_runtime': 100.0032, 'train_samples_per_second': 2.1, 'train_steps_per_second': 1.05, 'total_flos': 109742653440000.0, 'train_loss': 2.568669627961658})

In [ ]:
import os
save_directory = "./fine_tuned_gpt2"
os.makedirs(save_directory, exist_ok=True)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Fine-tuning complete. Model saved to:", save_directory)

Fine-tuning complete. Model saved to: ./fine_tuned_gpt2


In [ ]:
import os
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer from disk
model_dir = "./fine_tuned_gpt2"
tokenizer = GPT2LMHeadModel.from_pretrained(model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
model = GPT2LMHeadModel.from_pretrained(model_dir)

# Set pad token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Interactive generation loop
print("\nInteractive Generation Mode:")
print("Type your prompt (e.g., 'Math Riddle:') and the model will generate a math riddle with its solution.")
print("Type 'quit' to exit.\n")

while True:
    prompt = input("Enter a prompt (or type 'quit' to exit): ")
    if prompt.lower() == "quit":
        break

    # Tokenize the prompt and move it to the device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate text using the fine-tuned model with controlled generation parameters
    outputs = model.generate(
        **inputs,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=10,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\nGenerated Riddle with Solution:")
    print(generated_text)
    print("-" * 50)



Interactive Generation Mode:
Type your prompt (e.g., 'Math Riddle:') and the model will generate a math riddle with its solution.
Type 'quit' to exit.

Enter a prompt (or type 'quit' to exit): riddle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Riddle with Solution:
riddle: What is the ratio of 1 to 10? Solution: 10 = 1.6. Solution to
--------------------------------------------------
Enter a prompt (or type 'quit' to exit): riddle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Riddle with Solution:
riddle: What is the probability of a given number being a positive integer? Solution: 0.6.
--------------------------------------------------
Enter a prompt (or type 'quit' to exit): riddle


...


Generated Riddle with Solution:
riddle: I am a man. I have two children. My wife and I are both in my late
--------------------------------------------------
Enter a prompt (or type 'quit' to exit): riddle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Riddle with Solution:
riddle: How do I get a random number? Solution: I am a number between 0 and 1.
--------------------------------------------------
Enter a prompt (or type 'quit' to exit): riddle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Riddle with Solution:
riddle: What is the number of digits in a number? Solution: 7. Solution in decimal places.
--------------------------------------------------
Enter a prompt (or type 'quit' to exit): quit
